In [2]:
import numpy as np
import os
import boto3
import tensorflow as tf

In [3]:
import sagemaker
from sagemaker import get_execution_role

sagemaker_session = sagemaker.Session()

# Get a SageMaker-compatible role used by this Notebook Instance.
role = get_execution_role()

In [4]:
bucket = 'demo-saeed'
prefix = 'fraudcredit-keras'
dataset_name = 'creditcard.csv'

In [5]:
boto3.Session().resource('s3').Bucket(bucket).Object(os.path.join(prefix, 'data/train.csv')).upload_file('data/creditcard.csv')


In [6]:
# train_input = sagemaker_session.upload_data('data', key_prefix="{}/{}/{}".format(bucket_name, prefix, 'data') )
# train_input

In [7]:
data_location = 's3://{}/{}/{}'.format(bucket, prefix, 'data',dataset_name)
s3_input_train = sagemaker.s3_input(s3_data = data_location, content_type='csv')
s3_input_train.config

{'DataSource': {'S3DataSource': {'S3DataType': 'S3Prefix',
   'S3Uri': 's3://demo-saeed/fraudcredit-keras/data',
   'S3DataDistributionType': 'FullyReplicated'}},
 'ContentType': 'csv'}

In [8]:
train_input = 's3://demo-saeed/fraudcredit-keras/data/train.csv'
train_input

's3://demo-saeed/fraudcredit-keras/data/train.csv'

In [9]:
import pandas as pd
df = pd.read_csv(train_input, sep=',',  nrows=1000)
df.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


In [10]:
from sagemaker.tensorflow import TensorFlow

tf_estimator = TensorFlow(entry_point='main.py', 
                          role=role,
                          train_instance_count=1, 
                          train_instance_type='local',
                          framework_version='1.12', 
                          py_version='py3',
                          script_mode=True,
                          hyperparameters={'epochs': 1}
                         )  


In [11]:
tf_estimator.fit({'train': s3_input_train})

Creating tmpeikeq3o0_algo-1-h7wg0_1 ... 
Attaching to tmpeikeq3o0_algo-1-h7wg0_12mdone
algo-1-h7wg0_1  | 2020-03-08 18:56:14,363 sagemaker-containers INFO     Imported framework sagemaker_tensorflow_container.training
algo-1-h7wg0_1  | 2020-03-08 18:56:14,369 sagemaker-containers INFO     No GPUs detected (normal if no gpus installed)
algo-1-h7wg0_1  | 2020-03-08 18:56:14,540 sagemaker-containers INFO     No GPUs detected (normal if no gpus installed)
algo-1-h7wg0_1  | 2020-03-08 18:56:14,554 sagemaker-containers INFO     No GPUs detected (normal if no gpus installed)
algo-1-h7wg0_1  | 2020-03-08 18:56:14,563 sagemaker-containers INFO     Invoking user script
algo-1-h7wg0_1  | 
algo-1-h7wg0_1  | Training Env:
algo-1-h7wg0_1  | 
algo-1-h7wg0_1  | {
algo-1-h7wg0_1  |     "additional_framework_parameters": {},
algo-1-h7wg0_1  |     "channel_input_dirs": {
algo-1-h7wg0_1  |         "train": "/opt/ml/input/data/train"
algo-1-h7wg0_1  |     },
algo-1-h7wg0_1  |     "current_host": "algo-1-h7

In [12]:
tf_estimator.model_dir

's3://sagemaker-us-east-1-079329190341/sagemaker-tensorflow-scriptmode-2020-03-08-18-55-29-802/model'

# Prediciton using endpoint high-level sdk

In [175]:
import time

tf_endpoint_name = 'keras-tf-fmnist-'+time.strftime("%Y-%m-%d-%H-%M-%S", time.gmtime())

#tf_predictor = tf_estimator.deploy(initial_instance_count=1,
#                                   instance_type='ml.p2.xlarge')      # $1.361/hour in eu-west-1

tf_predictor = tf_estimator.deploy(initial_instance_count=1,
                         instance_type='local',        # $0.134/hour in eu-west-1
                         #accelerator_type='ml.eia1.medium',  # + $0.140/hour in eu-west-1
                         endpoint_name=tf_endpoint_name)     # = 80% discount!

Attaching to tmp6n1kw9a9_algo-1-gccm2_1
algo-1-gccm2_1  | INFO:__main__:starting services
algo-1-gccm2_1  | INFO:__main__:using default model name: model
algo-1-gccm2_1  | INFO:__main__:tensorflow serving model config: 
algo-1-gccm2_1  | model_config_list: {
algo-1-gccm2_1  |   config: {
algo-1-gccm2_1  |     name: "model",
algo-1-gccm2_1  |     base_path: "/opt/ml/model",
algo-1-gccm2_1  |     model_platform: "tensorflow"
algo-1-gccm2_1  |   }
algo-1-gccm2_1  | }
algo-1-gccm2_1  | 
algo-1-gccm2_1  | 
algo-1-gccm2_1  | INFO:__main__:nginx config: 
algo-1-gccm2_1  | load_module modules/ngx_http_js_module.so;
algo-1-gccm2_1  | 
algo-1-gccm2_1  | worker_processes auto;
algo-1-gccm2_1  | daemon off;
algo-1-gccm2_1  | pid /tmp/nginx.pid;
algo-1-gccm2_1  | error_log  /dev/stderr info;
algo-1-gccm2_1  | 
algo-1-gccm2_1  | worker_rlimit_nofile 4096;
algo-1-gccm2_1  | 
algo-1-gccm2_1  | events {
algo-1-gccm2_1  |   worker_connections 2048;
algo-1-gccm2_1  | }
algo-1-gccm2_1  | 
algo-1-gccm2_1  

In [19]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
RANDOM_SEED = 42
LABELS = ["Normal", "Fraud"]
data = df.drop(['Time'], axis=1)

data['Amount'] = StandardScaler().fit_transform(data['Amount'].values.reshape(-1, 1))
X_train, X_test = train_test_split(data, test_size=0.2, random_state=RANDOM_SEED)
X_train = X_train[X_train.Class == 0]
X_train = X_train.drop(['Class'], axis=1)

y_test = X_test['Class']
X_test = X_test.drop(['Class'], axis=1)

batch_file = 'batch_data.csv'
X_test.to_csv(batch_file,index=False,header=False)
#sess.upload_data(batch_file, key_prefix='{}/batch'.format(prefix))
boto3.Session().resource('s3').Bucket(bucket).Object(os.path.join(prefix, 'batch/batch_data.csv')).upload_file('batch_data.csv')

X_train = X_train.values
X_test = X_test.values
print(X_train.shape)

(798, 29)


In [14]:
type(X_test)

numpy.ndarray

In [15]:
X_test.shape

(200, 29)

In [16]:
# tt = X_test[0:2,]
# payload = json.dumps({'inputs':tt.tolist()})
# payload
# tf_predictor.predict(payload)

In [17]:
#tf_predictor.predict({'instances': X_test})

In [180]:
predictions = tf_predictor.predict(X_test)['predictions']

algo-1-gccm2_1  | 2020/03/06 21:53:14 [warn] 18#18: *2 an upstream response is buffered to a temporary file /var/cache/nginx/proxy_temp/1/00/0000000001 while reading upstream, client: 172.18.0.1, server: , request: "POST /invocations HTTP/1.1", subrequest: "/v1/models/model:predict", upstream: "http://127.0.0.1:8501/v1/models/model:predict", host: "localhost:8080"
algo-1-gccm2_1  | 172.18.0.1 - - [06/Mar/2020:21:53:14 +0000] "POST /invocations HTTP/1.1" 200 37731 "-" "-"


In [181]:
mse = np.mean(np.power(X_test - predictions, 2), axis=1)
error_df = pd.DataFrame({'reconstruction_error': mse,
                        'true_class': y_test})

In [182]:
error_df.head(20)

,reconstruction_error,true_class
521,0.225959,0
737,0.088699,0
740,0.612147,0
660,0.138237,0
411,0.190693,0
678,0.221506,0
626,0.123476,0
513,1.319569,0
859,0.161115,0
136,0.763574,0


# Prediciton using Tensorflow Serving

In [18]:
!pwd

/home/ec2-user/SageMaker/Saeed-demo/Credit-Card-Fraud-Detection-using-Autoencoders-in-Keras


In [20]:
!aws s3 cp s3://sagemaker-us-east-1-079329190341/sagemaker-tensorflow-scriptmode-2020-03-06-16-23-30-671/model.tar.gz model.tar.gz

download: s3://sagemaker-us-east-1-079329190341/sagemaker-tensorflow-scriptmode-2020-03-06-16-23-30-671/model.tar.gz to ./model.tar.gz


In [22]:
!tar -xf model.tar.gz

In [24]:
sess = tf.compat.v1.Session()
loaded = tf.saved_model.load(export_dir='SavedModels/autoencodermodel/1/', tags={'serve'},sess=sess )
loaded.signature_def

INFO:tensorflow:Restoring parameters from SavedModels/autoencodermodel/1/variables/variables


{'serving_default': inputs {
  key: "inputs"
  value {
    name: "input_1_1:0"
    dtype: DT_FLOAT
    tensor_shape {
      dim {
        size: -1
      }
      dim {
        size: 29
      }
    }
  }
}
outputs {
  key: "dense_4_1/Relu:0"
  value {
    name: "dense_4_1/Relu:0"
    dtype: DT_FLOAT
    tensor_shape {
      dim {
        size: -1
      }
      dim {
        size: 29
      }
    }
  }
}
method_name: "tensorflow/serving/predict"
}

In [26]:
!saved_model_cli show --dir SavedModels/autoencodermodel/1/ --tag_set serve --all



MetaGraphDef with tag-set: 'serve' contains the following SignatureDefs:

signature_def['serving_default']:
  The given SavedModel SignatureDef contains the following input(s):
    inputs['inputs'] tensor_info:
        dtype: DT_FLOAT
        shape: (-1, 29)
        name: input_1_1:0
  The given SavedModel SignatureDef contains the following output(s):
    outputs['dense_4_1/Relu:0'] tensor_info:
        dtype: DT_FLOAT
        shape: (-1, 29)
        name: dense_4_1/Relu:0
  Method name is: tensorflow/serving/predict


In [35]:
!nohup tensorflow_model_server \
  --rest_api_port=8501 \
  --model_name=mobilenet1 \
  --model_base_path="SavedModels/autoencodermodel/1" >server.log 2>&1

In [36]:
X_test

array([[ 0.07193637, -1.60199934, -0.6061928 , ..., -0.0895    ,
         0.09558394,  2.71879153],
       [ 1.18948241,  0.23724716,  0.24498043, ...,  0.02357161,
         0.01612106, -0.30105079],
       [-0.84808655, -0.57344051,  1.4913986 , ..., -0.39218566,
        -0.52142946,  0.74976351],
       ...,
       [ 0.95127828, -0.65772959,  1.70188353, ...,  0.10681691,
         0.04088053, -0.04503868],
       [-0.38672597,  0.17256499,  0.73258264, ...,  0.11404772,
         0.13568659, -0.06106615],
       [-0.57152075,  1.07160045,  1.28011025, ...,  0.07164145,
        -0.17551047, -0.30260011]])

In [37]:
!pip install -q requests

You should consider upgrading via the 'pip install --upgrade pip' command.


In [32]:

# import json
# import numpy
# import requests
# data = json.dumps({"signature_name": "serving_default",
#                    "instances": X_test.tolist()})
# headers = {"content-type": "application/json"}
# json_response = requests.post('http://localhost:8501/v1/models/mobilenet1:predict',
#                               data=data, headers=headers)
# predictions = numpy.array(json.loads(json_response.text)["predictions"])

In [38]:
!docker ps

CONTAINER ID        IMAGE               COMMAND             CREATED             STATUS              PORTS               NAMES


In [111]:
%%bash
mkdir -p tmp/tfserving
cd tmp/tfserving
git clone --depth=1 https://github.com/tensorflow/serving

Cloning into 'serving'...


In [112]:
%%bash
cd tmp/tfserving
docker pull tensorflow/serving:latest

latest: Pulling from tensorflow/serving
Digest: sha256:ea44bf657f8cff7b07df12361749ea94628185352836bb08065345f5c8284bae
Status: Image is up to date for tensorflow/serving:latest


In [113]:
!pwd

/home/ec2-user/SageMaker/Saeed-demo/Credit-Card-Fraud-Detection-using-Autoencoders-in-Keras


In [39]:
!docker run --rm -p 8501:8501 \
    --mount type=bind,source=$(pwd),target=$(pwd) \
    -e MODEL_BASE_PATH=$(pwd)/SavedModels \
    -e MODEL_NAME=autoencodermodel -t tensorflow/serving:latest

Unable to find image 'tensorflow/serving:latest' locally
latest: Pulling from tensorflow/serving

a4a261c9: Pulling fs layer 
20cdee96: Pulling fs layer 
60e1d0de: Pulling fs layer 
7668deea: Pulling fs layer 
b5699598: Pulling fs layer 
8f5dbe31: Pulling fs layer 
011e11a2: Pulling fs layer 
075f0126: Pull complete  280B/280BMBBBDigest: sha256:ea44bf657f8cff7b07df12361749ea94628185352836bb08065345f5c8284bae
Status: Downloaded newer image for tensorflow/serving:latest
2020-03-08 19:04:51.882717: I tensorflow_serving/model_servers/server.cc:86] Building single TensorFlow model file config:  model_name: autoencodermodel model_base_path: /home/ec2-user/SageMaker/Saeed-demo/Credit-Card-Fraud-Detection-using-Autoencoders-in-Keras/SavedModels/autoencodermodel
2020-03-08 19:04:51.882852: I tensorflow_serving/model_servers/server_core.cc:462] Adding/updating models.
2020-03-08 19:04:51.882864: I tensorflow_serving/model_servers/server_core.cc:573]  (Re-)adding model: autoencodermodel
2020-03-0

In [41]:
!docker ps
#!docker kill 96c5dc662795

CONTAINER ID        IMAGE                       COMMAND                  CREATED             STATUS              PORTS                              NAMES
aaf66c759b30        tensorflow/serving:latest   "/usr/bin/tf_serving…"   28 seconds ago      Up 23 seconds       8500/tcp, 0.0.0.0:8501->8501/tcp   optimistic_neumann


In [42]:
%%bash

curl http://localhost:8501/v1/models/autoencodermodel

{
 "model_version_status": [
  {
   "version": "1",
   "state": "AVAILABLE",
   "status": {
    "error_code": "OK",
    "error_message": ""
   }
  }
 ]
}


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   154  100   154    0     0  11000      0 --:--:-- --:--:-- --:--:-- 11000


In [43]:
!curl -d '{"instances": [[1.0,2.0,5.0,2.0,0.0, 1.0,2.0,5.0,2.0,0.0, 1.0,2.0,5.0,2.0,0.0, 1.0,2.0,5.0,2.0,0.0, 1.0,2.0,5.0,2.0,0.0, 1.0,2.0,5.0,2.0], \
[1.0,2.0,5.0,2.0,0.0, 1.0,2.0,5.0,2.0,0.0, 1.0,2.0,5.0,2.0,0.0, 1.0,2.0,5.0,2.0,0.0, 1.0,2.0,5.0,2.0,0.0, 1.0,2.0,5.0,2.0]]}' -X POST http://localhost:8501/v1/models/autoencodermodel:predict

{
    "predictions": [[1.0715723, 0.0, 2.18216205, 7.8084, 0.76329577, 2.31510925, 0.0, 0.0, 0.0, 0.53235352, 1.37354612, 0.0, 0.0, 0.0, 0.901465476, 0.440022677, 1.00044906, 0.0, 1.07490849, 0.0, 0.0, 0.0377479494, 0.0969904065, 0.0, 0.0, 0.800467908, 0.0, 0.0, 0.0], [1.0715723, 0.0, 2.18216205, 7.8084, 0.76329577, 2.31510925, 0.0, 0.0, 0.0, 0.53235352, 1.37354612, 0.0, 0.0, 0.0, 0.901465476, 0.440022677, 1.00044906, 0.0, 1.07490849, 0.0, 0.0, 0.0377479494, 0.0969904065, 0.0, 0.0, 0.800467908, 0.0, 0.0, 0.0]
    ]
}

In [44]:
data = '{"instances": [[1.0,2.0,5.0,2.0,0.0, 1.0,2.0,5.0,2.0,0.0, 1.0,2.0,5.0,2.0,0.0, 1.0,2.0,5.0,2.0,0.0, 1.0,2.0,5.0,2.0,0.0, 1.0,2.0,5.0,2.0], \
[1.0,2.0,5.0,2.0,0.0, 1.0,2.0,5.0,2.0,0.0, 1.0,2.0,5.0,2.0,0.0, 1.0,2.0,5.0,2.0,0.0, 1.0,2.0,5.0,2.0,0.0, 1.0,2.0,5.0,2.0]]}'
# files = {
#     'image_file': open('/home/hellouser/Downloads/infer/Users/User01/Images/tiles/999/00.jpg', 'rb')
# }

url = 'http://localhost:8501/v1/models/autoencodermodel:predict'
r = requests.post(url=url, data=data)

In [46]:
print(r.text)

{
    "predictions": [[1.0715723, 0.0, 2.18216205, 7.8084, 0.76329577, 2.31510925, 0.0, 0.0, 0.0, 0.53235352, 1.37354612, 0.0, 0.0, 0.0, 0.901465476, 0.440022677, 1.00044906, 0.0, 1.07490849, 0.0, 0.0, 0.0377479494, 0.0969904065, 0.0, 0.0, 0.800467908, 0.0, 0.0, 0.0], [1.0715723, 0.0, 2.18216205, 7.8084, 0.76329577, 2.31510925, 0.0, 0.0, 0.0, 0.53235352, 1.37354612, 0.0, 0.0, 0.0, 0.901465476, 0.440022677, 1.00044906, 0.0, 1.07490849, 0.0, 0.0, 0.0377479494, 0.0969904065, 0.0, 0.0, 0.800467908, 0.0, 0.0, 0.0]
    ]
}


# Prediction using Batch Transform

In [195]:
output_path='s3://{}/{}/batch_output/'.format(bucket, prefix)
output_path

's3://demo-saeed/fraudcredit-keras/batch_output/'

In [193]:
sm_transformer = tf_estimator.transformer(1, 'local', output_path='s3://{}/{}/batch_output/'.format(bucket, prefix))
# start a transform job
input_location = 's3://{}/{}/batch/{}'.format(bucket, prefix, 'batch_data.csv') # use input data without ID column
sm_transformer.transform(input_location, content_type='text/csv', split_type='Line')
sm_transformer.wait()

Attaching to tmppenc80r9_algo-1-y7998_1
algo-1-y7998_1  | INFO:__main__:starting services
algo-1-y7998_1  | INFO:__main__:using default model name: model
algo-1-y7998_1  | INFO:__main__:tensorflow serving model config: 
algo-1-y7998_1  | model_config_list: {
algo-1-y7998_1  |   config: {
algo-1-y7998_1  |     name: "model",
algo-1-y7998_1  |     base_path: "/opt/ml/model",
algo-1-y7998_1  |     model_platform: "tensorflow"
algo-1-y7998_1  |   }
algo-1-y7998_1  | }
algo-1-y7998_1  | 
algo-1-y7998_1  | 
algo-1-y7998_1  | INFO:__main__:nginx config: 
algo-1-y7998_1  | load_module modules/ngx_http_js_module.so;
algo-1-y7998_1  | 
algo-1-y7998_1  | worker_processes auto;
algo-1-y7998_1  | daemon off;
algo-1-y7998_1  | pid /tmp/nginx.pid;
algo-1-y7998_1  | error_log  /dev/stderr info;
algo-1-y7998_1  | 
algo-1-y7998_1  | worker_rlimit_nofile 4096;
algo-1-y7998_1  | 
algo-1-y7998_1  | events {
algo-1-y7998_1  |   worker_connections 2048;
algo-1-y7998_1  | }
algo-1-y7998_1  | 
algo-1-y7998_1  

In [217]:
import json
import io
from urllib.parse import urlparse

def get_csv_output_from_s3(s3uri, file_name):
    parsed_url = urlparse(s3uri)

    bucket_name = parsed_url.netloc

    prefix = parsed_url.path[1:]

    s3 = boto3.resource('s3')
    obj = s3.Object(bucket_name, '{}/{}'.format(prefix, file_name))

    return obj.get()["Body"].read().decode('utf-8')

In [240]:
output = get_csv_output_from_s3('s3://demo-saeed/fraudcredit-keras/batch_output/sagemaker-tensorflow-scriptmode-2020-03-2020-03-06-22-03-18-147', 'batch_data.csv.out')
j =json.loads(output)
n =j['predictions']
df = pd.DataFrame.from_records(n)
df.head()
#output_df = pd.read_csv(j['predictions'], sep=",", header=None)
# output_df.head(8)

,0,1,2,3,4,5,6,7,8,9,...,19,20,21,22,23,24,25,26,27,28
0,0.00000,0.000000,0.000000,1.61166,0.000000,0.000000,1.260130,0.000000,0.058937,0.0,...,0.0,0.0,0.000000,0.0,0.000000,0.00000,0.063055,0.0,0.0,2.951820
1,1.02960,0.226272,0.324172,1.03954,0.000000,0.000000,0.000000,0.000000,0.193701,0.0,...,0.0,0.0,0.000000,0.0,0.000000,0.19246,0.000000,0.0,0.0,0.000000
2,0.00000,0.000000,1.318160,0.00000,0.789449,0.109775,1.252820,0.362922,0.000000,0.0,...,0.0,0.0,0.299968,0.0,0.000000,0.00000,0.000000,0.0,0.0,0.520986
3,1.08857,0.141208,0.205083,0.00000,0.000000,0.000000,0.000000,0.000000,0.210598,0.0,...,0.0,0.0,0.000000,0.0,0.081515,0.07221,0.000000,0.0,0.0,0.000000
4,0.00000,0.000000,2.534320,0.00000,0.000000,0.000000,0.203742,0.106238,0.076987,0.0,...,0.0,0.0,0.328653,0.0,0.342895,0.00000,0.000000,0.0,0.0,0.000000


In [194]:
#!docker kill $(docker ps -q)

In [192]:
#!docker rm $(docker ps -a -q)

5de8b9368b8f
d52325faf8c3
e52b55e755a7
166773913f56
1d5df8bb1af1
8d13dd2e4878
5c9c67930d9a
